In [54]:
import re

import pandas as pd
import numpy as np 

In [55]:
def sammeforesatt( rec1, rec2 ): 
    """
    Sjekker om det er overlapp på navn, telefonnummer eller epost for to oppføringer fra "Foresatt rapport m epost" 
    
    Sjekker om det er overlapp på navn, telefonnummer eller epost for to oppføringer 
    fra "Foresatt rapport m epost" fra styreweb.com.
    
    ARGUMENTS
        rec1, rec2 : To pandas.series objekter hentet fra dataframe lest inn fra styreweb-regnearket
    
    KEYWORDS
        None 
        
    RETURNS 
        None eller dictionary
        Med matc så returnerer vi dictionary med navn på de to som (potensielt) er søsken, 
        dvs overlappende kontaktinfo på foresatte 
        None  
    """
    
    treffliste = []
    
    # Treff på etternavn
    if rec1['Etternavn'].lower() == rec2['Etternavn'].lower():
        treffliste.append( 'Etternavn')

    # Navn foresatt 1 og 2 er identisk
    if sjekknavn( rec1['Foresatt 1'], rec2['Foresatt 1'] ): 
        treffliste.append( 'Foresatt 1')
    if sjekknavn( rec1['Foresatt 1'], rec2['Foresatt 2'] ): 
        treffliste.append( 'Foresatt 1 og 2')
    if sjekknavn( rec1['Foresatt 2'], rec2['Foresatt 1'] ): 
        treffliste.append( 'Foresatt 1 og 2')
    if sjekknavn( rec1['Foresatt 2'], rec2['Foresatt 2'] ): 
        treffliste.append( 'Foresatt 2')
        
    if  rec1['e-post']   == rec2['e-post']    or \
        rec1['e-post']   == rec2['e-post.1']  or \
        rec1['e-post.1'] == rec2['e-post']    or \
        rec1['e-post.1'] == rec2['e-post.1']:
        
        treffliste.append( 'e-post')
        
    if len( treffliste) > 0: 
        return { 
            'Medlem 1' : ', '.join( [ rec1['Etternavn'], rec1['Fornavn'] ] ), 
            'Medlem 2' : ', '.join( [ rec2['Etternavn'], rec2['Fornavn'] ] ), 
        #    'Foresatt medlem 1' : ', '.join( sorted( [rec1['Foresatt 1'], 
        #                                              rec1['Foresatt 2'] ] ) ),
        #    'Foresatt medlem 2' : ', '.join( sorted( [rec2['Foresatt 1'], 
        #                                              rec2['Foresatt 2'] ] ) ), 
            'Treff på:'  : ', '.join( sorted( treffliste ))
        }
    else: 
        return None
        

def sjekknavn( navn1, navn2): 
    """
    Sjekker om deler av navn1 er lik deler av navn 2
    """
    tf = False 
    if isinstance( navn1, str ) and isinstance( navn2, str): 
        n1 = sorted(re.sub('[^a-zA-Z.\d\s]', '', navn1.lower()).split())
        n2 = sorted(re.sub('[^a-zA-Z.\d\s]', '', navn2.lower()).split())

        tf = all( [ x in n2 for x in n1  ] )
    
    return tf
    

In [56]:
mydf = pd.read_excel( 'Foresatt_Rapport_m_e-post.xlsx', header=2)
mydf = mydf[ ~mydf['Etternavn'].isnull()]
mydf = mydf[ ~mydf['Etternavn'].str.contains( 'Generert av')]
mydf.dtypes

Etternavn      object
Fornavn        object
Foresatt 1     object
Tlf           float64
Mobil         float64
e-post         object
Foresatt 2     object
Tlf.1         float64
Mobil.1       float64
e-post.1       object
dtype: object

In [57]:
rec1 = mydf.iloc[30]

In [58]:
rec2 = mydf.iloc[53]

In [59]:
r = sammeforesatt( rec1, rec2)